In [1]:
import plasmapy

Note that plasmapy can take a very long time to import, about a minute.

In [13]:
import hii_utils
import plasmapy.formulary as pf
import astropy.units as u

## Calculate various parameters for a typical H II region

### Fiducial parameters

As with my other notes, we will normalise to an electron density of 100 pcc and temperature of 10,000 K. We set the size scale to 10 pc (later, we can try and relate this to the ionization parameter)

In [3]:
L = 10.0 * u.pc
n = 100.0 * u.cm ** -3
T = 1.0e4 * u.K
species = ("p+", "p+")

### Magnetic field

Set the magnetic field to try and give a plasma beta of unity. Note that the `n` parameter in the `beta()` function is the total particle density, which is $2 n_e$ if fully ionized pure hydrogen.

In [25]:
B = 83.3* u.microgauss
pf.beta(T, 2*n, B)

<Quantity 1.00014539>

That is close enough to unity. But note that it is slightly different from the 78.2 microG that I derived in my notes, which is probably due to the correction for Helium, which I am ignoring here.

### Collisional quantities

In [4]:
pf.Knudsen_number(L, T, n, species)

<Quantity 1.79932417e-10>

Mean free path comes out the same for electrons and protons

In [5]:
pf.mean_free_path(T, n, species).cgs

<Quantity 5.55213426e+09 cm>

But it is slightly different for Helium ions

In [6]:
pf.mean_free_path(T, n, ("He++", "p+")).cgs

<Quantity 1.43652977e+09 cm>

And it is a *lot* bigger for high-velocity protons (for example, from stellar wind)

In [7]:
pf.mean_free_path(T, n, species, V=1000 * u.km/u.s).cgs

<Quantity 3.66291304e+16 cm>

By default, the transport coefficients are calculated in the Braginskii model

In [8]:
pf.ion_viscosity(T, n, T, n, "p")

<Quantity [1.07673276e-07, 1.07345943e-07, 1.07345943e-07, 0.00000000e+00,
           0.00000000e+00] Pa s>

But there is the option to use the more accurate Ji and Held (2013) model, although in our case this makes little difference.

But we can also a more recent theory from Ji & Held (2013), which changes the numbers very slightly. The main difference is that it better accounts for the electron-ion collisions, and apparently it has more effect in the magnetic case, which we will  see below. 

In [9]:
pf.ion_viscosity(T, n, T, n, "p", model="Ji-Held")

<Quantity [1.08256591e-07, 1.08256591e-07, 1.08256591e-07, 0.00000000e+00,
           0.00000000e+00] Pa s>

Note that viscosity is a tensor, which may become important once we introduce a magnetic field.

### Anisotropy with magnetic field

In [28]:
pf.Hall_parameter(n, T, B, "p", "p")

<Quantity 6708.50398785>

The Hall parameter is the product of Larmor frequency and collision time: $x = \omega_L \, \tau = (V / r_L)\, (\lambda / V) = \lambda / r_L$. so it is the ratio of the mean free path to the Larmor radius. This is very similar to the magnetic Prandtl number, which I calculated in the notes. It is linearly proportional to the B field:

In [30]:
pf.Hall_parameter(n, T, 0.1 * B, "p", "p")

<Quantity 670.85039879>

In [26]:
pf.ion_viscosity(T, n, T, n, "p", B=B, model="Ji-Held")

<Quantity [1.08256591e-07, 1.66132734e-23, 6.64530909e-23, 1.24610698e-15,
           2.49221389e-15] Pa s>

In [31]:
pf.ion_viscosity(T, n, T, n, "p", B=B)

<Quantity [1.07673276e-07, 1.66132734e-23, 6.64530913e-23, 1.24610698e-15,
           2.49221389e-15] Pa s>

These are now the tensor viscosity resuts with a more realistic magnetic field